In [46]:
import pdfplumber as pdf
import pandas as pd
import numpy as np

In [3]:
pdf_tw = pdf.open("./pdf/tw.pdf")
pdf_eu = pdf.open("./pdf/35to129_eu.pdf")

### Taiwan


In [79]:
tw_pdf = []
with pdf_tw as pdf:
    pages = pdf.pages
    for page in pages:
        page = page.extract_table()
        for table in page:
            tw_pdf.append(table)

tw_pdf = pd.DataFrame(data=tw_pdf[1:-1], columns=tw_pdf[0]).drop(["No."], axis=1)
tw_pdf = tw_pdf.drop_duplicates(keep=False)
tw_pdf = tw_pdf.reset_index(drop=True)

tw_pdf.replace(to_replace="-", value="NO CAS", regex=False, inplace=True)
tw_pdf.rename(columns={"Chemical name": "Chemical Name"}, inplace=True)

tw_pdf = tw_pdf.assign(**{"CAS No.": tw_pdf["CAS No."].str.split("\n")})

# Explode the "CAS No." column to create separate rows for each CAS number
tw_pdf = tw_pdf.explode("CAS No.")

# Remove leading/trailing whitespace from the "CAS No." column
tw_pdf["CAS No."] = tw_pdf["CAS No."].str.strip()

# Drop duplicates
tw_pdf = tw_pdf.drop_duplicates()

tw_pdf["CAS No."] = tw_pdf["CAS No."].replace(to_replace="/", value="", regex=True)

# Reset the index
tw_pdf = tw_pdf.reset_index(drop=True)

tw_pdf

,Chemical Name,CAS No.
0,Mercury and its compounds (with the exception ...,7439-97-6
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2
2,4-Benzyloxyphenol and 4-ethoxyphenol,622-62-8
3,Bithionol,97-18-7
4,Pilocarpine and its salts,92-13-7
...,...,...
732,Raw material made from bovine and sheep tissue...,NO CAS
733,Alanroot oil（Inula helenium L.）,97676-35-2
734,"Rauwolfia serpentina L., alkaloids and their s...",90106-13-1
735,Yohimbine and its salts,146-48-5


In [80]:
print(tw_pdf.iloc[577])

Chemical Name    Coumetarol (INN)
CAS No.                 4366-18-1
Name: 577, dtype: object


### EU


In [68]:
eu_pdf = []
with pdf_eu as pdf:
    pages = pdf.pages
    for page in pages:
        page = page.extract_table()
        for table in page:
            eu_pdf.append(table)
eu_pdf = pd.DataFrame(data=eu_pdf[1:], columns=eu_pdf[0])
eu_pdf = eu_pdf.drop_duplicates(keep=False).reset_index(drop=True)

In [77]:
eu_pdf.rename(
    columns={"Chemical name/INN": "Chemical Name", "CAS number": "CAS No."},
    inplace=True,
)

on9space = eu_pdf["CAS No."].iloc[15]

eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(
    to_replace=on9space, value="NO CAS", regex=False
)

eu_pdf = eu_pdf.assign(**{"CAS No.": eu_pdf["CAS No."].str.split("\n")})

# Explode the "CAS No." column to create separate rows for each CAS number
eu_pdf = eu_pdf.explode("CAS No.")

# Remove leading/trailing whitespace from the "CAS No." column
eu_pdf["CAS No."] = eu_pdf["CAS No."].str.strip()

# Drop duplicates
eu_pdf = eu_pdf.drop_duplicates()

eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(to_replace="/", value="", regex=True)
eu_pdf.replace(to_replace="—", value="NO CAS", regex=False, inplace=True)
eu_pdf.replace(to_replace="\n", value=" ", regex=True, inplace=True)
eu_pdf["CAS No."] = eu_pdf["CAS No."].replace("\\[[^\\]]*\\]", "", regex=True)

another7space = eu_pdf["CAS No."].iloc[1868]
eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(another7space, "NO CAS", regex=True)

# Reset the index
eu_pdf = eu_pdf.reset_index(drop=True)

eu_pdf.tail(50)

,Chemical Name,CAS No.
1827,Vinyl acetate,108-05-4
1828,Aclonifen (ISO); 2-chloro-6-nitro-3-phenoxyani...,74070-46-5
1829,"2-Ethylhexyl 10-ethyl-4,4-dimethyl-7-oxo-8-oxa...",57583-35-4
1830,Dimethyltin dichloride,753-73-1
1831,4-Vinylcyclohexene,100-40-3
1832,Tralkoxydim (ISO); 2-(N-ethoxypropanimidoyl)-3...,87820-88-0
1833,Cycloxydim (ISO); 2-(N-ethoxybutanimidoyl)-3- ...,101205-02-1
1834,"Fluazinam (ISO); 3-chloro-N-[3-chloro-2,6-dini...",79622-59-6
1835,"Penconazole (ISO); 1-[2-(2,4-dichlorophenyl)pe...",66246-88-6
1836,Fenoxycarb (ISO); ethyl [2-(4-phenoxyphen­ oxy...,72490-01-8


### Comparison

on chemical names


In [81]:
comparison = pd.merge(
    tw_pdf["Chemical Name"], eu_pdf["Chemical Name"], how="outer", indicator="Exist"
)
comparison.replace(to_replace="left_only", value="Taiwan", inplace=True)
comparison.replace(to_replace="right_only", value="EU", inplace=True)
comparison.replace(to_replace="both", value="Both", inplace=True)
comparison

,Chemical Name,Exist
0,Mercury and its compounds (with the exception ...,Taiwan
1,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,4-Benzyloxyphenol and 4-ethoxyphenol,Both
4,4-Benzyloxyphenol and 4-ethoxyphenol,Both
...,...,...
2304,Reaction products of paraformaldehyde with 2- ...,EU
2305,Methylhydrazine,EU
2306,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
2307,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


on CAS no.


In [111]:
cas_comp = tw_pdf.merge(
    eu_pdf,
    on="CAS No.",
    how="outer",
    indicator="CAS Exist",
    suffixes=("(Taiwan)", "(EU)"),
)
cas_comp.replace(to_replace="left_only", value="Taiwan", inplace=True)
cas_comp.replace(to_replace="right_only", value="EU", inplace=True)
cas_comp.replace(to_replace="both", value="Both", inplace=True)
cas_comp

,Chemical Name(Taiwan),CAS No.,Chemical Name(EU),CAS Exist
0,Mercury and its compounds (with the exception ...,7439-97-6,"Mercury and its compounds, except those specia...",Both
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,622-62-8,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,Bithionol,97-18-7,Bithionol (INN),Both
4,Pilocarpine and its salts,92-13-7,Pilocarpine and its salts,Both
...,...,...,...,...
7634,NaN,3830-45-3,Nonadecafluorodecanoic acid [1] Ammonium nonad...,EU
7635,NaN,5625-90-1,"N,N′-Methylenedimorpholine; N,N′-methylenebism...",EU
7636,NaN,55219-65-3,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
7637,NaN,111988-49-9,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


In [112]:
cas_comp.drop(["CAS Exist"], axis=1, inplace=True)
cas_comp.reset_index(drop=True, inplace=True)
cas_comp.to_csv("./result/cas_comp.csv", index=False)

In [101]:
no_cas_excl = cas_comp.query("`CAS No.` != 'NO CAS'")
all_no_cas = cas_comp.query("`CAS No.` == 'NO CAS'")
no_cas_excl

,Chemical Name(Taiwan),CAS No.,Chemical Name(EU),CAS Exist
0,Mercury and its compounds (with the exception ...,7439-97-6,"Mercury and its compounds, except those specia...",Both
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,622-62-8,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,Bithionol,97-18-7,Bithionol (INN),Both
4,Pilocarpine and its salts,92-13-7,Pilocarpine and its salts,Both
...,...,...,...,...
7634,NaN,3830-45-3,Nonadecafluorodecanoic acid [1] Ammonium nonad...,EU
7635,NaN,5625-90-1,"N,N′-Methylenedimorpholine; N,N′-methylenebism...",EU
7636,NaN,55219-65-3,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
7637,NaN,111988-49-9,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


Just double checking if any error (NaNs):


In [85]:
comparison_nan = comparison[comparison["Exist"].isna()]
len(comparison_nan)

0

### Filter names - EU only, Taiwan only and both as 3 dataframes


In [106]:
comp_eu = (
    comparison.query("Exist == 'EU'").drop_duplicates(keep=False).reset_index(drop=True)
)
comp_eu

,Chemical Name,Exist
0,N-(5-Chlorobenzoxazol-2-yl)acetamide,EU
1,(2-Acetoxyethyl)trimethylammonium hydroxide (A...,EU
2,Aminocaproic acid (INN) and its salts,EU
3,Aconitine (principal alkaloid of Aconitum nape...,EU
4,"Alkyne alcohols, their esters, ethers and salts",EU
...,...,...
1162,Reaction products of paraformaldehyde with 2- ...,EU
1163,Methylhydrazine,EU
1164,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
1165,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


In [107]:
comp_tw = (
    comparison.query("Exist == 'Taiwan'")
    .drop_duplicates(keep=False)
    .reset_index(drop=True)
)
comp_tw

,Chemical Name,Exist
0,Mercury and its compounds (with the exception ...,Taiwan
1,Bithionol,Taiwan
2,Halogeno-salicylanilide,Taiwan
3,Sodium perborate,Taiwan
4,Sodium borate\nexcept for sodium borate used t...,Taiwan
...,...,...
279,"Phenazinium, 3,7-diamino-2,8-dimethyl-5-phenyl...",Taiwan
280,"Ethanaminium, N-[4-[[4-(diethylamino)phenyl][4...",Taiwan
281,Raw material made from bovine and sheep tissue...,Taiwan
282,Alanroot oil（Inula helenium L.）,Taiwan


In [108]:
comp_both = (
    comparison.query("Exist == 'Both'")
    .drop_duplicates(keep=False)
    .reset_index(drop=True)
)
comp_both

,Chemical Name,Exist
0,Pilocarpine and its salts,Both
1,"Cells, tissues or products of human origin",Both
2,Vinyl chloride monomer,Both
3,"2,2'-Dihydroxy-3,3',5,5',6,6'-hexachlorodiphen...",Both
4,Tretinoin (INN) (retinoic acid and its salts),Both
...,...,...
307,Thallium and its compounds,Both
308,Neodymium and its salts,Both
309,Tellurium and its compounds,Both
310,"Rauwolfia serpentina L., alkaloids and their s...",Both


### Export to excel


Taiwan count no cas no.
EU count no cas no.
Taiwan count cas no.
EU count cas no.

eu can get rid of all the same cas no. item
same with taiwan

eu whats left is the unique items
taiwan whats left is the unique items

in the case of having cas no.:
eu - both = eu unique
taiwan - both = taiwan unique

check whether eu unique is in taiwan unique


In [102]:
with pd.ExcelWriter("./result/25oct_comparison.xlsx") as writer:
    comparison.to_excel(writer, sheet_name="Names Full Comparison")
    comp_eu.to_excel(writer, sheet_name="Only in EU(Names)")
    comp_tw.to_excel(writer, sheet_name="Only in Taiwan(Names)")
    comp_both.to_excel(writer, sheet_name="Exist in Both(Names)")
    cas_comp.to_excel(writer, sheet_name="CAS No. Comparison")
    all_no_cas.to_excel(writer, sheet_name="Everything without CAS No.")
    no_cas_excl.to_excel(writer, sheet_name="Everything with CAS No.")